# Process field-1 and field-2 images


The field trial images were cut into plots using the "extract_plots.sh" bash script.

The functions described here are used for:
* Converting images from GeoTiff to numpy arrays.
* Rotating the images.
* Export as numpy array and JPEG.
* Segmentation of soil and plant pixels using a vegetation index.
* Use the bounding box labels for discriminating between crop and weed pixels.
* Exporting the images and segmentation masks.

In [ ]:
import os
import numpy as np
import cv2
#from osgeo import gdal
import matplotlib.pyplot as plt
import shutil
import pandas as pd
import json
import random
%matplotlib inline

from glob import glob

In [ ]:
# Load Field trials
drn = glob("/group/pawsey0149/mmckay/singularity/Drone_norepeats/*")
c180731 = glob("/group/pawsey0149/mmckay/singularity/T1_Cund_WR_180731/*")
c180903 = glob("/group/pawsey0149/mmckay/singularity/T1_Cund_WR_180903/*")
t1m1 = glob("/group/pawsey0149/mmckay/singularity/t1mimagecoco1/*")
t1m2 = glob("/group/pawsey0149/mmckay/singularity/t1mimagecoco2/*")
t1m3 = glob("/group/pawsey0149/mmckay/singularity/t1mimagecoco3/*")
t1m4 = glob("/group/pawsey0149/mmckay/singularity/t1mimagecoco4/*")
t2m1 = glob("/group/pawsey0149/mmckay/singularity/t2mimagecoco1/*")
t2m2 = glob("/group/pawsey0149/mmckay/singularity/t2mimagecoco2/*")
t2m3 = glob("/group/pawsey0149/mmckay/singularity/t2mimagecoco3/*")
t2m4 = glob("/group/pawsey0149/mmckay/singularity/t2mimagecoco4/*")
t2m5 = glob("/group/pawsey0149/mmckay/singularity/t2mimagecoco5/*")
t2m6 = glob("/group/pawsey0149/mmckay/singularity/t2mimagecoco6/*")

In [ ]:
yc = glob("/group/pawsey0149/mmckay/singularity/YC/*")

In [ ]:
yc

# Take a look at the images

In [ ]:
a = cv2.cvtColor(cv2.imread(drn[1]), cv2.COLOR_BGR2RGB)
b = cv2.cvtColor(cv2.imread(t1m1[1]), cv2.COLOR_BGR2RGB)

In [ ]:
# band 
fig, axs = plt.subplots(2,2, figsize=(10, 10), facecolor='w', edgecolor='k')
axs = axs.ravel()

title = ['ft 1', 'ft 2']
plotted =[a,b]

for i in range(2):
    im = plotted[i]
    axs[i].hist(im[0])
    axs[i+2].imshow(im)
    axs[i].set_title(f"Image from: {title[i]} with {im.shape}")
#worth choosing 10 random images with plots for thesis

# Labels and masks

The polygon labels were drawn using Makesense.ai and exported as COCO labels

## Segmentation of Plant - Soil

In [ ]:
drn[1]

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
#basically gets rid of soil and identifies plants
#drn
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result

#Soil masks for drn
otsu_thresh = []
for i in drn:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('Drone_norepeats', 'inputImages_Masks_drn')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:
#check image works
tt = np.load("/group/pawsey0149/mmckay/singularity/inputImages_Masks_drn/DJI_0001_soil_mask.npy")
plt.imshow(tt)

#show a few of these compared to normal images

In [ ]:
c180731[1]

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
#c180731
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for c180731
otsu_thresh = []
for i in c180731:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('T1_Cund_WR_180731', 'inputImages_Masks_c180731')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')


In [ ]:
#check image works
tt2 = np.load("/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/Cund_T1_1001_soil_mask.npy")
plt.imshow(tt2)

In [ ]:
c180903[1]

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for c180903
otsu_thresh = []
for i in c180903:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('T1_Cund_WR_180903', 'inputImages_Masks_c180903')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:
#check image works
tt3 = np.load("/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180903/IMG_0808_soil_mask.npy")
plt.imshow(tt3)

In [ ]:
t1m1

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for t1m1
otsu_thresh = []
for i in t1m1:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('t1mimagecoco1', 'inputImages_Masks_t1m1')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for t1m2
otsu_thresh = []
for i in t1m2:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('t1mimagecoco2', 'inputImages_Masks_t1m2')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for t1m3
otsu_thresh = []
for i in t1m3:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('t1mimagecoco3', 'inputImages_Masks_t1m3')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for t1m4
otsu_thresh = []
for i in t1m4:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('t1mimagecoco4', 'inputImages_Masks_t1m4')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for t2m1
otsu_thresh = []
for i in t2m1:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('t2mimagecoco1', 'inputImages_Masks_t2m1')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for t2m2
otsu_thresh = []
for i in t2m2:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('t2mimagecoco2', 'inputImages_Masks_t2m2')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')


In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for t2m3
otsu_thresh = []
for i in t2m3:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('t2mimagecoco3', 'inputImages_Masks_t2m3')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')


In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for t2m4
otsu_thresh = []
for i in t2m4:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('t2mimagecoco4', 'inputImages_Masks_t2m4')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for t2m5
otsu_thresh = []
for i in t2m5:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('t2mimagecoco5', 'inputImages_Masks_t2m5')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for t2m6
otsu_thresh = []
for i in t2m6:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('t2mimagecoco6', 'inputImages_Masks_t2m6')
    filename = os.path.basename(i).replace('.JPG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:
#YC
# Calculate the VI for masks - In this case it is CIVE
def create_soilplant_maks(im):
    CIVE = 0.441*im[:,:,0] - 0.881*im[:,:,1] + 0.385*im[:,:,2] +18.78745
    otsu_threshold, image_result = cv2.threshold(CIVE.astype('uint8'), 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return otsu_threshold, image_result


#Soil masks for yc
otsu_thresh = []
for i in yc:
    im = cv2.imread(i)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    otsu_threshold, image_result = create_soilplant_maks(im)
    path = os.path.dirname(i).replace('YC', 'inputImages_Masks_YC')
    filename = os.path.basename(i).replace('.PNG',"")
    if not os.path.exists(path):
        os.makedirs(path)
    # save the image as npy
    np.save(f"{path}/{filename}_image.npy", im)
    # save the soil mask as npy
    np.save(f'{path}/{filename}_soil_mask.npy', image_result)
    otsu_thresh.append(otsu_threshold)
    
a = pd.Series(otsu_thresh)
a.to_csv(f'{path}/otsu_threshold.csv')

In [ ]:

# My own figure, top is 4 random images, middle is 4 soil masks, bottom is 4 soild crop masks
# Make font identical to paper font
ft1mi= glob('/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/*image.npy')
ft1mm= glob('/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/*mask.npy')
import random

fig, axs = plt.subplots(2,4, figsize=(20,20))
axs = axs.ravel()

fig.suptitle("Plant and soil segmentation masks using CIVE", fontsize=35)

#import 4 random raw images
for i in range(4):
    filename = os.path.basename(random.choice(ft1mi))
    im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/{filename}')
    mask = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/{filename.replace("image", "soil_mask")}')
    soil_crop_mask = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/{filename.replace("image", "soil_crop_mask")}')
    axs[i].imshow(im)
    axs[i+4].imshow(mask,alpha=1)
    
    # some embelishments in the plot
    #
    title = filename.replace('.jpeg', '').replace("id_", "")
    axs[i+4].set_title(title, fontsize = 30)
    axs[i+4].axis("off")
    axs[i].set_title(title, fontsize = 30)
    axs[i].axis("off")

plt.tight_layout()





In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(20, 20))
axs = axs.ravel()

fig.suptitle("Plant and soil segmentation masks using CIVE", fontsize=35)

image_files = glob.glob('/group/pawsey0149/mmckay/singularity/soil_masks/*image.npy')
soil_mask_files = glob.glob('/group/pawsey0149/mmckay/singularity/soil_masks/*soil_mask.npy')
soil_crop_mask_files = glob.glob('/group/pawsey0149/mmckay/singularity/soil_masks/*soil_crop_mask.npy')

# Import and display 4 random raw images
for i in range(4):
    filename = random.choice(image_files)
    im = np.load(filename)
    axs[i].imshow(im)
    axs[i].set_title(filename, fontsize=30)
    axs[i].axis("off")

# Import and display 4 soil masks
for i in range(4):
    filename = random.choice(soil_mask_files)
    mask = np.load(filename)
    axs[i + 4].imshow(mask, alpha=1)
    axs[i + 4].set_title(filename, fontsize=30)
    axs[i + 4].axis("off")

# Import and display 4 soil crop masks
for i in range(4):
    filename = random.choice(soil_crop_mask_files)
    crop_mask = np.load(filename)
    axs[i + 8].imshow(crop_mask, alpha=1)
    axs[i + 8].set_title(filename, fontsize=30)
    axs[i + 8].axis("off")

plt.tight_layout()
plt.show()

## Segmentation Crop - Weed

In [ ]:
# Get the bounding box labels
#example from monica
#with open('/data/weed_data/manuallabel_makesense/F1_field_/labels_f1_fieldtrial_complete.json') as f:
    #ft1_labels =  json.load(f)
#ft1_images = sorted(glob("/data/weed_data/inputImages_Masks/F1_field/*_image.npy"))
#ft1_masks = sorted(glob("/data/weed_data/inputImages_Masks/F1_field/*_soil_mask.npy"))

# Loading my bounding box labels

In [ ]:
# Get the bounding box labels
#Dronelabels
with open('/group/pawsey0149/mmckay/singularity/Dronelabelscoco.json') as a:
    drn_labels =  json.load(a)
drn_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_drn/*_image.npy"))
drn_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_drn/*_soil_mask.npy"))


In [ ]:
#c180731 labels
with open('/group/pawsey0149/mmckay/singularity/t1cundwr180731labelscoco.json') as b:
    c180731_labels =  json.load(b)
c180731_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/*_image.npy"))
c180731_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/*_soil_mask.npy"))

In [ ]:
c180731_labels

In [ ]:
c180731_masks[1]

In [ ]:
#c180903 labels
with open('/group/pawsey0149/mmckay/singularity/Cundtwr180903coco.json') as c:
    c180903_labels =  json.load(c)
c180903_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180903/*_image.npy"))
c180903_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180903/*_soil_mask.npy"))

In [ ]:
#t1m1 labels
with open('/group/pawsey0149/mmckay/singularity/t1milinglabelscoco1.json') as d:
    t1m1_labels =  json.load(d)
t1m1_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/*_image.npy"))
t1m1_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/*_soil_mask.npy"))

In [ ]:
#t1m2 labels
with open('/group/pawsey0149/mmckay/singularity/t1milinglabelscoco2.json') as e:
    t1m2_labels =  json.load(e)
t1m2_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m2/*_image.npy"))
t1m2_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m2/*_soil_mask.npy"))

In [ ]:
#t1m3 labels
with open('/group/pawsey0149/mmckay/singularity/t1milinglabelscoco3.json') as f:
    t1m3_labels =  json.load(f)
t1m3_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m3/*_image.npy"))
t1m3_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m3/*_soil_mask.npy"))

In [ ]:
#t1m4 labels
with open('/group/pawsey0149/mmckay/singularity/t1milinglabelscoco4.json') as g:
    t1m4_labels =  json.load(g)
t1m4_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m4/*_image.npy"))
t1m4_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m4/*_soil_mask.npy"))

In [ ]:
#t2m1 labels
with open('/group/pawsey0149/mmckay/singularity/t2milinglabelscoco1.json') as h:
    t2m1_labels =  json.load(h)
t2m1_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m1/*_image.npy"))
t2m1_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m1/*_soil_mask.npy"))

In [ ]:
#t2m2 labels
with open('/group/pawsey0149/mmckay/singularity/t2milinglabelscoco2.json') as i:
    t2m2_labels =  json.load(i)
t2m2_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m2/*_image.npy"))
t2m2_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m2/*_soil_mask.npy"))

In [ ]:
#t2m3 labels
with open('/group/pawsey0149/mmckay/singularity/t2milinglabelscoco3.json') as j:
    t2m3_labels =  json.load(j)
t2m3_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m3/*_image.npy"))
t2m3_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m3/*_soil_mask.npy"))

In [ ]:
#t2m4 labels
with open('/group/pawsey0149/mmckay/singularity/t2milinglabelscoco4.json') as k:
    t2m4_labels =  json.load(k)
t2m4_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m4/*_image.npy"))
t2m4_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m4/*_soil_mask.npy"))

In [ ]:
#t2m5 labels
with open('/group/pawsey0149/mmckay/singularity/t2milinglabelscoco5.json') as l:
    t2m5_labels =  json.load(l)
t2m5_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m5/*_image.npy"))
t2m5_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m5/*_soil_mask.npy"))

In [ ]:
#t2m6 labels
with open('/group/pawsey0149/mmckay/singularity/t2milinglabelscoco6.json') as m:
    t2m6_labels =  json.load(m)
t2m6_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m6/*_image.npy"))
t2m6_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m6/*_soil_mask.npy"))

In [ ]:
#YC labels
with open('/group/pawsey0149/mmckay/singularity/YC_labels/labels_york_canola_2023-07-06-11-43-03.json') as m:
    yc_labels =  json.load(m)
yc_images = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/*_image.npy"))
yc_masks = sorted(glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/*_soil_mask.npy"))

In [ ]:
#functions for mask
#only reads in coco format
import PIL.Image
import PIL.ImageDraw
import skimage.io

#mask is produced using a blank np array which is then filled using the polygon bounding boxes converted to np array
def polygons_to_mask(img_shape, polygons):
    mask = np.zeros(img_shape, dtype=np.uint8)
    mask = PIL.Image.fromarray(mask)
    xy = list(map(tuple, polygons))
    PIL.ImageDraw.Draw(mask).polygon(xy=xy, outline=1, fill=1)
    mask = np.array(mask, dtype=bool)
    return mask
#create blank lists which ae populated 
def get_segmentation_coord(label_file, image_id=1):
    points=[]
    labels=[]
    half_mask=[]
    final_mask=[]
    
    anno = label_file['annotations']
    
    for ann in anno:
        im_id = ann['image_id']
        if im_id == image_id:
            seg1 = ann['segmentation']
            seg = seg1[0]
        
            for j in range(0,len(seg),2):
                x = seg[j]
                y = seg[j+1]
                mas = [x,y]
                half_mask.append(mas)
            final_mask.append(half_mask)
            half_mask=[]
        else:
            pass
    return final_mask

def weed_plant_labelling(soil_mask, weed_mask):
    final_mask = soil_mask.copy()
    final_mask[weed_mask] = np.where(final_mask[weed_mask] == 1, 2, 0)
    return final_mask
#bands refer to the 3 bands of colour in the image, r,g,b
#we are reading in an array of x*y*3 for each band
def remove_class(img, mask, obj_class=0):
    removed = np.zeros(img.shape, dtype='uint8')

    for i in range(img.shape[2]):
        band = img[:,:,i]
        band = np.where(mask == obj_class, 0, band)
        removed[:,:, i] = band
    return removed

In [ ]:
# Information needed to draw the weed mask
#drone crop mask
for i, im_label in enumerate(drn_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_drn/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_drn/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_drn/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(drn_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_drn/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

In [ ]:
# SANITY CHECKS
#drone crop mask
#add thrid row of figures, zoom in to one section
import random

# prepare Figure
fig, axs = plt.subplots(2,3, figsize=(50,50))
axs = axs.ravel()
fig.suptitle('Assessing mask precision in dataset F1')

for i, j in enumerate([0,2,4]):
    filename = os.path.basename(random.choice(drn_images)).replace('image.npy', "")
    baseimage = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_drn/{filename}image.npy")
    soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_drn/{filename}soil_mask.npy")
    weed_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_drn/{filename}soil_crop_mask.npy")
    
    # Plot the RGB overlaid with masks   
    axs[j].imshow(baseimage)
    b = remove_class(baseimage, soil_mask, obj_class=0)
    axs[1+j].imshow(b)
    axs[1+j].imshow(weed_mask, alpha=0.3)

    # some embelishments in the plot
    title = filename.replace('.npy', '').replace("id_", "")
    axs[j].set_title(title, fontsize=30)
    axs[1+j].set_title(title, fontsize=30)
    axs[j].axis("off")
    axs[1+j].axis("off")
    
plt.tight_layout()

In [ ]:
# Information needed to draw the weed mask
#c180731 crop mask
for i, im_label in enumerate(c180731_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(c180731_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

# Problems with c187031

In [ ]:
#to detect if no soil_crop_mask has been made for images
# load all the images "_image.npy"
#im_list = glob("/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/*_image.npy")
#replace_list =[]

#for i in im_list:
    # get the path for the weed_mask
#    filename = os.path.basename(i)
#    filename = filename.replace("image", "soil_crop_mask")
#     weed_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{filename}soil_crop_mask.npy")
#    if not os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{filename}'):
#        print(filename)
#        replace_list.append(filename)
        

In [ ]:
#to copy and save soil_mask as soil_crop_mask
#for images with no labels that do not create a soil mask
#Problems as we are saving the entire plant mask as the crop mask, therefore we are wrongly assigning weeds as crops
#for i in replace_list:
#    source = i
#    new = i.replace("soil_crop_mask", "soil_mask")
#    mask = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{new}')
#    np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{source}', mask)



In [ ]:
print(filename)

In [ ]:
#c180731 crop mask
#add thrid row of figures, zoom in to one section
#see section above for issues
import random

# prepare Figure
fig, axs = plt.subplots(2,4, figsize=(50,50))
axs = axs.ravel()
fig.suptitle('Assessing mask precision in dataset F1')

for i, j in enumerate([0,2,4]):
    filename = os.path.basename(random.choice(c180731_images)).replace('image.npy', "")
    baseimage = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{filename}image.npy")
    soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{filename}soil_mask.npy")
    weed_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180731/{filename}soil_crop_mask.npy")
    
    
    # Plot the RGB overlaid with masks   
    axs[j].imshow(baseimage)
    b = remove_class(baseimage, soil_mask, obj_class=0)
    axs[1+j].imshow(b)
    axs[1+j].imshow(weed_mask, alpha= 0.3)

    # some embelishments in the plot
    title = filename.replace('.npy', '').replace("id_", "")
    axs[j].set_title(title, fontsize=30)
    axs[1+j].set_title(title, fontsize=30)
    axs[j].axis("off")
    axs[1+j].axis("off")
    
plt.tight_layout()

In [ ]:
# Information needed to draw the weed mask
#c180903 crop mask
for i, im_label in enumerate(c180903_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180903/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180903/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180903/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(c180903_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_c180903/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass



In [ ]:
# Information needed to draw the weed mask
#t1m1 crop mask
for i, im_label in enumerate(t1m1_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(t1m1_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass


In [ ]:
#t1m1 crop mask figure
#add thrid row of figures, zoom in to one section
import random

# prepare Figure
fig, axs = plt.subplots(2,3, figsize=(50,50))
axs = axs.ravel()
fig.suptitle('Assessing mask precision in dataset F1')

for i, j in enumerate([0,2,4]):
    filename = os.path.basename(random.choice(t1m1_images)).replace('image.npy', "")
    baseimage = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/{filename}image.npy")
    soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/{filename}soil_mask.npy")
    weed_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m1/{filename}soil_crop_mask.npy")
    
    
    # Plot the RGB overlaid with masks   
    axs[j].imshow(baseimage)
    b = remove_class(baseimage, soil_mask, obj_class=0)
    axs[1+j].imshow(b)
    axs[1+j].imshow(weed_mask, alpha=0.3)

    # some embelishments in the plot
    title = filename.replace('.npy', '').replace("id_", "")
    axs[j].set_title(title, fontsize=30)
    axs[1+j].set_title(title, fontsize=30)
    axs[j].axis("off")
    axs[1+j].axis("off")
    
plt.tight_layout()

In [ ]:
# Information needed to draw the weed mask
#t1m2 crop mask
for i, im_label in enumerate(t1m2_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m2/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m2/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m2/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(t1m2_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m2/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass


In [ ]:
# Information needed to draw the weed mask
#t1m3 crop mask
for i, im_label in enumerate(t1m3_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m3/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m3/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m3/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(t1m3_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m3/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

In [ ]:
# Information needed to draw the weed mask
#t1m4 crop mask
for i, im_label in enumerate(t1m4_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m4/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m4/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m4/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(t1m4_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t1m4/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

In [ ]:
# Information needed to draw the weed mask
#t2m1 crop mask
for i, im_label in enumerate(t2m1_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m1/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m1/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m1/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(t2m1_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m1/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

In [ ]:
# Information needed to draw the weed mask
#t2m2 crop mask
for i, im_label in enumerate(t2m2_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m2/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m2/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m2/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(t2m2_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m2/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

In [ ]:
# Information needed to draw the weed mask
#t2m3 crop mask
for i, im_label in enumerate(t2m3_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m3/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m3/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m3/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(t2m3_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m3/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

In [ ]:
# Information needed to draw the weed mask
#t2m4 crop mask
for i, im_label in enumerate(t2m4_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m4/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m4/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m4/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(t2m4_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m4/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

In [ ]:
# Information needed to draw the weed mask
#t2m5 crop mask
for i, im_label in enumerate(t2m5_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m5/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m5/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m5/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(t2m5_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m5/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

In [ ]:
# Information needed to draw the weed mask
#t2m6 crop mask
for i, im_label in enumerate(t2m6_labels["images"]):
    filename = im_label["file_name"].replace(".JPG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m6/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m6/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m6/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(t2m6_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_t2m6/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

In [ ]:
# Information needed to draw the weed mask
#YC crop mask
for i, im_label in enumerate(yc_labels["images"]):
    filename = im_label["file_name"].replace(".PNG", "")
    im_shape = [im_label["height"], im_label["width"]]
    
    # load the image and soil mask
    if os.path.exists(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/{filename}_image.npy'):
        rgb_im = np.load(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/{filename}_image.npy')
        soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/{filename}_soil_mask.npy")
    
        # get coordenates
        im_id = im_label["id"]
        segcoord = get_segmentation_coord(yc_labels, image_id=im_id)
        mask1 = polygons_to_mask(im_shape, segcoord[0])
        for j in range(1, len(segcoord)):
            maskany = polygons_to_mask(im_shape,segcoord[j])
            mask1 += maskany
        
        final_mask1 = weed_plant_labelling(soil_mask, mask1)
        np.save(f'/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/{filename}_soil_crop_mask.npy', final_mask1)
    else:
        pass

In [ ]:
#YC crop mask figure
#add thrid row of figures, zoom in to one section
import random

# prepare Figure
fig, axs = plt.subplots(2,3, figsize=(50,50))
axs = axs.ravel()
fig.suptitle('Assessing mask precision in dataset F1')

for i, j in enumerate([0,2,4]):
    filename = os.path.basename(random.choice(yc_images)).replace('image.npy', "")
    baseimage = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/{filename}image.npy")
    soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/{filename}soil_mask.npy")
    weed_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/{filename}soil_crop_mask.npy")
    
    
    # Plot the RGB overlaid with masks   
    axs[j].imshow(baseimage)
    b = remove_class(baseimage, soil_mask, obj_class=0)
    axs[1+j].imshow(b)
    axs[1+j].imshow(weed_mask, alpha=0.3)

    # some embelishments in the plot
    title = filename.replace('.npy', '').replace("id_", "")
    axs[j].set_title(title, fontsize=30)
    axs[1+j].set_title(title, fontsize=30)
    axs[j].axis("off")
    axs[1+j].axis("off")
    
plt.tight_layout()

In [ ]:
import random

# prepare Figure
fig, axs = plt.subplots(2,3, figsize=(50,50))
axs = axs.ravel()
fig.suptitle('Assessing mask precision in dataset F1')

for i, j in enumerate([0,2,4]):
    filename = os.path.basename(random.choice(yc_images)).replace('image.npy', "")
    baseimage = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/{filename}image.npy")
    soil_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/{filename}soil_mask.npy")
    weed_mask = np.load(f"/group/pawsey0149/mmckay/singularity/inputImages_Masks_YC/{filename}soil_crop_mask.npy")
    
    # Plot the RGB overlaid with masks   
    axs[j].imshow(baseimage)
    b = remove_class(baseimage, soil_mask, obj_class=0)
    axs[1+j].imshow(b)
    axs[1+j].imshow(weed_mask, alpha=0.3)

    # some embelishments in the plot
    title = filename.replace('.npy', '').replace("id_", "")
    axs[j].set_title(title, fontsize=30)
    axs[1+j].set_title(title, fontsize=30)
    axs[j].axis("off")
    axs[1+j].axis("off")
    
plt.tight_layout()